### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None

### Import Excel Files

In [ ]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\2021 Weekly Closed Invoice\Closed_invoice_01_03_2021'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

### Generate Weekly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [ ]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_line = excel_list[excel]['Line']

    df_select_header = df_header[['Accounting_CustomerID', 'CustomerID', 'DocNo', 'Facility', 'Bill To', 'InvoiceDate']]
    df_drop_dup_header = df_select_header.drop_duplicates(subset = ['Accounting_CustomerID', 'DocNo',
                                                                    'Facility', 'Bill To'])
    
    df_select_line = df_line[['CustomerID', 'Facility', 'Bill Pay Category', 'Bill To']]
    df_drop_dup_line = df_select_line.drop_duplicates(subset = ['Facility', 'Bill To', 'Bill Pay Category'], 
                                                      keep = 'last')

    df_combine_header_line = pd.merge(df_drop_dup_line, df_drop_dup_header, how = 'left', 
                                      on = ['Facility', 'Bill To']).T.drop_duplicates().T
    df_combine_rename = df_combine_header_line.rename(columns = {"CustomerID_x": "CustomerID"})
    df_combine_week = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'DocNo', 'Facility', 
                                                                 'Bill Pay Category', 'Bill To', 'InvoiceDate'])
    print(df_combine_week.shape)
    
    df_list.append(df_combine_week)

### Generate Month Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [ ]:
df_combine_list = []
df_combine_month = df_list[0]

for df in range(len(df_list)):
    
    df_combine_month = pd.merge(df_combine_month, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'DocNo', 
                                        'Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_month.shape)
    
    df_combine_list.append(df_combine_month)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.T.drop_duplicates().T
df_output.rename({'InvoiceDate_x':'InvoiceDate', 'InvoiceDate_y':'InvoiceDate'}, axis = 'columns', inplace = True)

print(df_output.shape)

In [ ]:
ls1 = np.array(df_output.columns[0:6])

ls2 = np.array([])
for num in np.arange(1, 1 + len(df_output.columns[6:])):
    ls2 = np.append(ls2, str('Week ') + str(num))

week_name = np.append(ls1, ls2)
print(week_name)

df_output.columns = week_name

In [ ]:
df_output.head()

### Export Quarter Report As Excel File

In [ ]:
# df_output.to_excel("01_03_2021 Weekly Report.xlsx", index = False, header = True)